In [2]:
from pathlib import Path 
import os
import torch
from transformers import MarianMTModel, MarianTokenizer
import logging
from datetime import datetime
from torch.profiler import profile, record_function, ProfilerActivity, schedule

In [2]:
print(f'@@Configuration START@@')

@@Configuration START@@


In [3]:
# Define configuration 
batch_size = 128
d_model = 256
n_head = 8
max_len = 80
ffn_hidden = 512
n_layers=4
drop_prob=0.1
epochs=30
init_lr = 0.00 # having warmup step
eps = 5e-9
weight_decay = 5e-4
warmup_steps=1500
clip = 1

print(f'batch_size: {batch_size}')
print(f'd_model: {d_model}')
print(f'n_head: {n_head}') 
print(f'max_len: {max_len}') 
print(f'ffn_hidden: {ffn_hidden}')
print(f'n_layers: {n_layers}')
print(f'drop_prob: {drop_prob}')
print(f'epochs: {epochs}')
print(f'init_lr: {init_lr}')
print(f'weight_decay: {weight_decay}')
print(f'clip: {clip}')

batch_size: 64
d_model: 400
n_head: 8
max_len: 128
ffn_hidden: 1200
n_layers: 4
drop_prob: 0.1
epochs: 50
init_lr: 0.0
weight_decay: 0.0005
clip: 1


In [4]:
# Configure Device 


if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA as device")
else:
    # Check that MPS is available
    if not torch.backends.mps.is_available():
        if not torch.backends.mps.is_built():
            print("MPS not available because the current PyTorch install was not "
                  "built with MPS enabled.")
        else:
            print("MPS not available because the current MacOS version is not 12.3+ "
                  "and/or you do not have an MPS-enabled device on this machine.")
        device = torch.device("cpu")
        print("Using CPU as device")
    else:
        device = torch.device("mps")
        print("Using MPS as device")
    
torch.set_default_device(device)

Using MPS as device


In [5]:
# Define tokenizers
TOKENIZERS_PARALLELISM = True

kr_tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-ko-en')
en_tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-ko-en')

enc_voc_size = kr_tokenizer.vocab_size
dec_voc_size = en_tokenizer.vocab_size

print(f'Using kr_tokenizer: Helsinki-NLP/opus-mt-ko-en')
print(f'kr_tokenizer_voc_size(enc_voc_size): {enc_voc_size}')

print(f'Using en_tokenizer: Helsinki-NLP/opus-mt-ko-en')
print(f'en_tokenizer_voc_size(dec_voc_size): {dec_voc_size}')

# Define some variables that are going to be used in future
src_pad_token = kr_tokenizer.pad_token_id
src_eos_token = kr_tokenizer.eos_token_id

trg_pad_token = en_tokenizer.pad_token_id
trg_sos_token = en_tokenizer.bos_token_id
trg_eos_token = en_tokenizer.eos_token_id

print(f'src_pad_token: {src_pad_token}')
print(f'src_eos_token: {src_eos_token}')
print(f'trg_pad_token: {trg_pad_token}')
print(f'trg_sos_token: {trg_sos_token}')
print(f'trg_eos_token: {trg_eos_token}')

/Users/ball/Documents/workspace/transformer-tutorial/.venv/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Using kr_tokenizer: Helsinki-NLP/opus-mt-ko-en
kr_tokenizer_voc_size(enc_voc_size): 65001
Using en_tokenizer: Helsinki-NLP/opus-mt-ko-en
en_tokenizer_voc_size(dec_voc_size): 65001
src_pad_token: 65000
src_eos_token: 0
trg_pad_token: 65000
trg_sos_token: None
trg_eos_token: 0


In [7]:
# Define path configuration for the project 
project_dir = Path(os.getcwd()).parent
rawdata_dir = project_dir / "data"
data_dir = project_dir / "preprocessed"
model_dir = project_dir / "models"

rawdata_dir.mkdir(parents=True, exist_ok=True)
data_dir.mkdir(parents=True, exist_ok=True)
model_dir.mkdir(parents=True, exist_ok=True)

print(f'project_dir: {project_dir}')
print(f'rawdata_dir: {rawdata_dir}')
print(f'data_dir: {data_dir}')

project_dir: /Users/ball/Documents/workspace/transformer-tutorial
rawdata_dir: /Users/ball/Documents/workspace/transformer-tutorial/data
data_dir: /Users/ball/Documents/workspace/transformer-tutorial/preprocessed


In [8]:
# Define learning rate scheduler. 
# If you want to modify the logic of Scheduler, please modify this class 

class LRScheduler:
    def __init__(self, optimizer, d_model, warmup_steps, LR_scale=1): 
        self.optimizer = optimizer
        self.step_count = 0 
        self.d_model = d_model
        self.warmup_steps = warmup_steps
        self.LR_scale = LR_scale
        self._d_model_factor = self.LR_scale * (self.d_model ** -0.5)
    def step(self): 
        self.step_count += 1 
        lr = self.calculate_learning_rate()
        self.optimizer.param_groups[0]['lr'] = lr 
    def calculate_learning_rate(self): 
        minimum_factor = min(self.step_count ** -0.5, self.step_count * self.warmup_steps ** -1.5)
        return self._d_model_factor * minimum_factor

In [9]:
# Configure Logger 
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

logging_dir = project_dir / "logs" 

logging_dir.mkdir(parents=True, exist_ok=True)

log_file = logging_dir / f'log_{timestamp}.log'

logger = logging.getLogger('transformer_log')
logger.setLevel(logging.INFO)

file_handler = logging.FileHandler(log_file)
file_handler.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)

logger.addHandler(file_handler)

In [ ]:
# For profiling 
profiler_schedule = schedule(wait=1, warmup=1, active=1, repeat=1)
profiler_activities = [ProfilerActivity.CPU, ProfilerActivity.CUDA]
profiler_record_shapes=True
profiler_profile_memory=True


In [10]:
print(f'@@Configuration END@@')

@@Configuration END@@
